## Mathematical inverse problems

1. $Ax = b$, b is known (data) and we are looking for $A^{-1}$ to get the original $x$.
2. During the task of reconstruction we are given $b$ (the projections) and we are looking for methods to get some form of method to approximate $A^{-1}$
3. There are many problems with $A^{-1}$, it is impossible to model explicitly and the problem (1) is ill-positioned in the Hadamard sense. So we are looking for methods, which approximately/closely solve the problem of (1).

## Reconstruction methods

The most known method is the filtered-backprojection (FBP), which is based on the inverse Radon-transform as follows.

### Filtered back-projection

#### The forward transformation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale

image = shepp_logan_phantom()
image = rescale(image, scale=0.4, mode='reflect', channel_axis=None)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5))

ax1.set_title("Original")
ax1.imshow(image, cmap=plt.cm.Greys_r)

theta = np.linspace(0.0, 180.0, max(image.shape), endpoint=False)
sinogram = radon(image, theta=theta)
dx, dy = 0.5 * 180.0 / max(image.shape), 0.5 / sinogram.shape[0]
ax2.set_title("Radon transform\n(Sinogram)")
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax2.imshow(
    sinogram,
    cmap=plt.cm.Greys_r,
    extent=(-dx, 180.0 + dx, -dy, sinogram.shape[0] + dy),
    aspect='auto',
)

fig.tight_layout()
plt.show()

#### The backward transformation and filters

In [ ]:
import matplotlib.pyplot as plt
from skimage.transform.radon_transform import _get_fourier_filter

filters = ['ramp', 'shepp-logan', 'cosine', 'hamming', 'hann']

for ix, f in enumerate(filters):
    response = _get_fourier_filter(2000, f)
    plt.plot(response, label=f)

plt.xlim([0, 1000])
plt.xlabel('frequency')
plt.legend()
plt.show()

In [ ]:
from skimage.transform import iradon

reconstruction_fbp = iradon(sinogram, theta=theta, filter_name='ramp')
error = reconstruction_fbp - image
print(f'FBP rms reconstruction error: {np.sqrt(np.mean(error**2)):.3g}')

imkwargs = dict(vmin=-0.2, vmax=0.2)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5), sharex=True, sharey=True)
ax1.set_title("Reconstruction\nFiltered back projection")
ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax2.set_title("Reconstruction error\nFiltered back projection")
ax2.imshow(reconstruction_fbp - image, cmap=plt.cm.Greys_r, **imkwargs)
plt.show()

The problem with FBP are the following:
* Results are statistically not correct for SPECT data, which is $\lambda$-Poisson
* Can not be used with attenuation maps (CT) for attenuation correction
* Good for mathematically correct estimations for calibration measurements

In [ ]:
# now specialize it for SPECT data and do the following tasks:
# Load the SPECT projection data and plot them as they are in the dicom file
# Reconstruct the the volume (original distribution) from the projection frames

### Maximum-likelihood Expectation Maximization (MLEM)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#for the Shepp-Logan phantom, Radon-transform, resize...
from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale, resize

import math
import time

In [ ]:
# creation of MLEM datastructures and steps to run the reconstruction
image_f = shepp_logan_phantom()
image_f = resize(image_f, (3,3) )

image_f *= 255.0/image_f.max()

#print('image_f min, max: ', np.min(image_f), np.max(image_f))

image_g = radon(image_f)

#print('image_g min, max: ', np.min(image_g), np.max(image_g))

# plt.imshow(image_g)
# plt.show()

g = image_g.copy()
f = image_f.copy()

g_row, g_col = g.shape[:]
f_row, f_col = f.shape[:]

#print('g shape: ', g_row, g_col)
#print('f shape: ', f_row, f_col)
#print('')

#1.feladat. Az A mátrix meghatározsa

gs = np.reshape(g,(g.shape[0]*g.shape[1], 1))
fs = np.reshape(f,(f.shape[0]*f.shape[1], 1))

#print('gs shape: ', gs.shape)
#print('fs shape: ', fs.shape)
#print('')

A = np.zeros((gs.shape[0], fs.shape[0]),dtype=float)

N, angles_num = g.shape[:]

angles = np.linspace(0, math.pi, angles_num) # it is recommended

COS = np.zeros((N,len(angles)))
SIN = np.zeros((N,len(angles)))

x = np.linspace(-0.5,0.5,N)

for k in range(0,N):
    for a in range(0,len(angles)):
        COS[k,a] = x[k]*math.cos(angles[a])
        SIN[k,a] = x[k]*math.sin(angles[a])

f_num = 0

for i in range(0, N):
    for j in range(0, N):
        for angle in range(0,len(angles)):
            r = COS[i,angle] + SIN[j,angle]
            index = (r + 0.5)*N
            if index >= 0 and index < N:
                ind = math.floor(index)
                if  (index - math.floor(index)) > 0.5:
                    A[ind    , f_num] = 1 - (index - math.floor(index))
                    A[ind + 1, f_num] = (index - math.floor(index))
                else:
                    index = math.floor(index)
                    A[ind, f_num] = (index - math.floor(index))
                    A[ind + 1, f_num] = 1 - (index - math.floor(index))
        f_num = f_num + 1

#print('A shape: ', A.shape)
# print('')
#
f0 = fs.copy()
f1 = f0.copy()

for i in range(0, f0.shape[0]):
    if f0[i] < 0:
        f0[i] = 0

#number of pixels
m = fs.shape[0]

#number of bins
n = gs.shape[0]

In [ ]:
# running the actual reconstruction
start = time.time()

for k in range(0,10):
    for j in range(0, fs.shape[0]):
        Sa = 0
        for i in range(0, n):
            Sa = Sa + A[i, j]

        S = 0
        for i in range(0, n):
            Saf = 0
            for j_v in range(0, m):
                Saf = Saf + A[i,j_v] * f0[j_v]
            if Saf != 0:
                S = S + gs[i] * A[i, j] / Saf
        if Sa != 0:
            f1[j] = S * f0[j] / Sa
        #print('Iteration (j):', j)
    f0 = f1
    print('+++++++++++++++++++++++++++++')
    print('Iteration: ', k)

end = time.time()
print('Elapsed time: ',(end - start)/60)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ff = np.reshape(f0,(f.shape[0], f.shape[1]))
#print(np.max(f), np.min(f))

ff *= 255.0/ff.max()

print('ff min, max: ', np.min(ff), np.max(ff))

print(f)
print('+++++++++++++++++++++++++++++++++++++++++++++')
print(ff)
#print(image_f)

ax1.imshow(int(f), cmap='gray')
ax2.imshow(int(ff), cmap='gray')

plt.show()

### Oredered Subsets Expectation Maximization (OSEM)